<a href="https://colab.research.google.com/github/mixmaxze/modelagem_topicos_lda/blob/main/T%C3%B3picos_com_LDA_em_s%C3%A1tiras_do_Sensacionalista_Jos%C3%A9_Gama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelagem de Tópicos com LDA (Alocação latente de Dirichlet)

### Quando descrevemos textos, frequentemente o fazemos usando além das palavras do texto também quais são os assuntos ou **tópicos** de que um texto trata. LDA descobre grupos de palavras que são utilizadas em conjunto e aproxima esses tópicos.

### Feito por José Matheus do Nascimento Gama para a disciplina de Processamento de Linguagem Natural (Ciência da Computação - UFCG).

In [ ]:
import altair as alt
import nltk
import numpy as np
import pandas as pd
import re

from nltk.corpus import stopwords
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Importando os dados

df = pd.read_csv("https://raw.githubusercontent.com/caiolibanio/atividade_NLP/master/csv_satiras_politicas.csv")
df.head()

,title,text,label
0,crise e tao grande que nem tiozao do pave fez piada na noite de natal,a familia guimaraes passou a noite de natal perplexa. genival o tiozao do pave deles desde 1985 nao fez a tradicional piada quando as sobremesas da ceia chegaram. dispensou a piada e o pave. limitouse a se servir de pudim de leite e sentar no sofa mudo. tambem nao contou piadas sexistas e pa...,satire
1,nao me representam diz jesus sobre intolerantes que apedrejaram crianca que saia do candomble,uma menina de 11 anos apedrejada ao sair de uma festa de candomble no ultimo domingo no rio de janeiro fez a sociedade parar e refletir sobre a intolerancia. apedrejada por homens que gritaram vai queimar no inferno a menina hoje diz ter medo de morrer. mas nao foram apenas comentaristas deste m...,satire
2,marina silva e heloisa helena montam novo partido o partido esquerdista da republica unida p. e. r. u,insatisfeitas com seus partidos com as siglas dos partidos com os politicos com as tardes ensolaradas com os dias chuvosos com a lei da gravidade marina silva e heloisa helena resolveram se unir ate ficarem tambem insatisfeitas uma com a outra e montar um novo partido o partido esquerdista da re...,satire
3,dez propostas que podem realmente mudar o brasil,o instituto nupal nucleo de pesquisas da america latina fez uma pesquisa que mostrou ideias que podem mudar o brasil de verdade. eis as propostas1 diminuicao do numero de zeros nos boletos. 2 proibicao de ouvir funk alto dentro de onibus e outros coletivos. 3 multa para piadas do pave. 4 multa ...,satire
4,apresentadora do cidade alerta bahia dara curso de como ser otimista e feliz,assassinatos sequestros mortes violentas. nenhuma noticia parece abalar a apresentadora do cidade alerta da bahia mariana sena. ao fim de cada noticiario capaz de abalar as mentes mais equilibradas do planeta ela sempre sorri e diz que esta se sentindo otima. ponto para mariana que no mes que...,satire


## Pré-processamento

In [ ]:
# Definindo as stop words (língua portuguesa) + outras específicas

stop_words = set(stopwords.words("portuguese"))
stop_words.update(['adeus', 'agora', 'aí', 'ainda', 'além', 'algo', 'alguém', 'algum', 'alguma', 'algumas', 'alguns', 'ali', 'ampla', 'amplas', 'amplo', 'amplos', 'ano', 'anos', 'ante', 'antes', 'apenas', 'apoio', 'após', 'aqui', 'área', 'assim', 'atrás', 'através', 'baixo', 'bastante', 'bem', 'boa', 'boas', 'bom', 'bons', 'breve', 'cá', 'cada', 'catorze', 'cedo', 'cento', 'certamente', 'certeza', 'cima', 'cinco', 'coisa', 'coisas', 'conselho', 'contra', 'contudo', 'custa', 'dá', 'dão', 'daquela', 'daquelas', 'daquele', 'daqueles', 'dar', 'debaixo', 'demais', 'dentro', 'desde', 'dessa', 'dessas', 'desse', 'desses', 'desta', 'destas', 'deste', 'destes', 'deve', 'devem', 'devendo', 'dever', 'deverá', 'deverão', 'deveria', 'deveriam', 'devia', 'deviam', 'dez', 'dezanove', 'dezasseis', 'dezassete', 'dezoito', 'dia', 'diante', 'disse', 'disso', 'disto', 'dito', 'diz', 'dizem', 'dizer', 'dois', 'doze', 'duas', 'dúvida', 'embora', 'enquanto', 'és', 'estar', 'estás', 'estiveste', 'estivestes', 'etc', 'exemplo', 'faço', 'falta', 'favor', 'faz', 'fazeis', 'fazem', 'fazemos', 'fazendo', 'fazer', 'fazes', 'feita', 'feitas', 'feito', 'feitos', 'fez', 'fim', 'final', 'forma', 'foste', 'fostes', 'geral', 'grande', 'grandes', 'grupo', 'havia', 'hoje', 'hora', 'horas', 'la', 'lá', 'lado', 'lo', 'local', 'logo', 'longe', 'lugar', 'maior', 'maioria', 'mal', 'máximo', 'meio', 'menor', 'menos', 'mês', 'meses', 'mesma', 'mesmas', 'mesmos', 'mil', 'momento', 'muita', 'muitas', 'muitos', 'nada', 'naquela', 'naquelas', 'naquele', 'naqueles', 'nenhum', 'nenhuma', 'nessa', 'nessas', 'nesse', 'nesses', 'nesta', 'nestas', 'neste', 'nestes', 'ninguém', 'nível', 'noite', 'nome', 'nova', 'novas', 'nove', 'novo', 'novos', 'número', 'nunca', 'obra', 'obrigada', 'obrigado', 'oitava', 'oitavo', 'oito', 'onde', 'ontem', 'onze', 'outra', 'outras', 'outro', 'outros', 'parece', 'parte', 'partir', 'paucas', 'pequena', 'pequenas', 'pequeno', 'pequenos', 'per', 'perante', 'perto', 'pode', 'pude', 'pôde', 'podem', 'podendo', 'poder', 'poderia', 'poderiam', 'podia', 'podiam', 'põe', 'põem', 'pois', 'ponto', 'pontos', 'porém', 'porque', 'porquê', 'posição', 'possível', 'possivelmente', 'posso', 'pouca', 'poucas', 'pouco', 'poucos', 'primeira', 'primeiras', 'primeiro', 'primeiros', 'própria', 'próprias', 'próprio', 'próprios', 'próxima', 'próximas', 'próximo', 'próximos', 'pude', 'puderam', 'quais', 'quáis', 'quanto', 'quantos', 'quarta', 'quarto', 'quatro', 'quê', 'quer', 'quereis', 'querem', 'queremas', 'queres', 'quero', 'questão', 'quinta', 'quinto', 'quinze', 'relação', 'sabe', 'sabem', 'segunda', 'segundo', 'sei', 'seis', 'sempre', 'sendo', 'ser', 'sete', 'sétima', 'sétimo', 'sexta', 'sexto', 'si', 'sido', 'sim', 'sistema', 'sob', 'sobre', 'sois', 'tal', 'talvez', 'tampouco', 'tanta', 'tantas', 'tanto', 'tão', 'tarde', 'têm', 'tendes', 'tendo', 'tens', 'ter', 'terceira', 'terceiro', 'ti', 'tido', 'tiveste', 'tivestes', 'toda', 'todas', 'todavia', 'todo', 'todos', 'trabalho', 'três', 'treze', 'tudo', 'última', 'últimas', 'último', 'últimos', 'umas', 'uns', 'vai', 'vais', 'vão', 'vários', 'vem', 'vêm', 'vendo', 'vens', 'ver', 'vez', 'vezes', 'viagem', 'vindo', 'vinte', 'vir', 'vós', 'vossa', 'vossas', 'vosso', 'vossos', 'zero'])

# Limpando os textos

clean_satires = []

for s in range(len(df['text'])):

    satire = df['text'].iloc[s]

    # Remove caracteres especiais 
    # Não irei tirar os dígitos pois talvez as datas sejam importantes nessas sátiras 
    satire = re.sub("(\\d|\\W)+|\w*\d\w*"," ", satire)

    clean_satires.append(satire)

clean_satires[0:6]

['a familia guimaraes passou a noite de natal perplexa genival o tiozao do pave deles desde nao fez a tradicional piada quando as sobremesas da ceia chegaram dispensou a piada e o pave limitouse a se servir de pudim de leite e sentar no sofa mudo tambem nao contou piadas sexistas e passou a noite sem qualquer trocadilho idiota e a crise servidor publico estadual genival esta sem receber teve que vender seu chevette e nao conseguiu comprar presentes para nenhum dos sobrinhos a noite de natal sem a piada do pave ou pacume nao e a mesma diz rafael pereira anos um de seus sobrinhos esperamos que ano que vem as coisas melhores e ele volte a fazer essa piada tao idiota porque noite de natal sem piada do pave e pior do que sem rabanada ',
 'uma menina de anos apedrejada ao sair de uma festa de candomble no ultimo domingo no rio de janeiro fez a sociedade parar e refletir sobre a intolerancia apedrejada por homens que gritaram vai queimar no inferno a menina hoje diz ter medo de morrer mas nao

### LDA funciona baseado em frequências de palavras, então usaremos TFs, e não TF-IDFs.

In [ ]:
# COUNT_VECTORIZER

tf_vectorizer = CountVectorizer(
        min_df = 30,
        max_df = 0.5,
        max_features = 10000,
        stop_words = stop_words,
        ngram_range = (1,2)
)

# Transforma

vec_satires = tf_vectorizer.fit_transform(clean_satires)

# Retorna uma lista de palavras

words = tf_vectorizer.get_feature_names_out()

print(vec_satires.shape)
print(len(words))

(4374, 1703)
1703


In [ ]:
words[600:620]

array(['expresidente', 'expresidente luiz', 'expresidente lula',
       'expressao', 'exterior', 'extremamente', 'fa', 'faca', 'facebook',
       'facil', 'faixa', 'fala', 'falam', 'falando', 'falar', 'falei',
       'falou', 'fama', 'familia', 'familias'], dtype=object)

## Encontrando tópicos

O resultado terá 

*   uma matriz que descreve a relação entre palavras e tópicos
*   uma matriz que descreve a relação entre documentos e tópicos

In [ ]:
# Funções auxiliares

def print_top_words(model, feature_names, n_top_words):
  for topic_idx, topic in enumerate(model.components_):
    print("\n--\nTopic #{}: ".format(topic_idx + 1))
    message = ", ".join([feature_names[i]
                          for i in topic.argsort()[:-n_top_words - 1:-1]])
    print(message)
  print()

def display_topics(W, H, feature_names, documents, no_top_words, no_top_documents):
    for topic_idx, topic in enumerate(H):
        print(f"\n--\nTopic #{topic_idx + 1}: ")
        print(", ".join([feature_names[i]
                for i in topic.argsort()[:-no_top_words - 1:-1]]).upper())
        top_d_idx = np.argsort(W[:,topic_idx])[::-1][0:no_top_documents]
        for d in top_d_idx: 
          doc_data = df['title'].iloc[d]
          print('{}: \t{:.2f}'.format(doc_data[0:35], W[d, topic_idx]))

In [ ]:
# Aplicando o LDA

lda = LatentDirichletAllocation(n_components=8, 
                                learning_method='online', # 'online' equivale a minibatch no k-means
                                random_state=0)

lda.fit(vec_satires)
doc_topic_matrix = lda.transform(vec_satires)

In [ ]:
print('Matriz documento-tópicos:' + str(doc_topic_matrix.shape))
print('Matriz tópicos-termos:' + str(lda.components_.shape))

Matriz documento-tópicos:(4374, 8)
Matriz tópicos-termos:(8, 1703)


In [ ]:
# Exibe os tópicos e os títulos das sátiras associadas a cada tópico

display_topics(doc_topic_matrix,
               lda.components_, 
               words,
               df,
               15, 
               10)


--
Topic #1: 
TEMER, PRESIDENTE, BRASIL, MICHEL, GOVERNO, MICHEL TEMER, JA, PAIS, TAMBEM, BRASILEIROS, PESSOAS, FACEBOOK, BRASILEIRO, SAO, APOS
 se quem pergunta por que o povo na: 	0.98
 reforma trabalhista e aprovada e t: 	0.98
 cego agradece a deus por nao poder: 	0.97
 medium diz que niemeyer esta com m: 	0.96
 para tentar aumentar popularidade : 	0.86
 o que marcela temer pensou na hora: 	0.86
 temer vai terminar de queimar seu : 	0.86
 dilma garante que o novo filme do : 	0.84
 temer vai dar consultoria sobre go: 	0.83
 para aumentar popularidade temer d: 	0.82

--
Topic #2: 
RIO, JANEIRO, JA, RIO JANEIRO, CIDADE, APOS, CACOFONIAS, VINICIUS, ANTUNES, PREFEITO, TAMBEM, SO, VINICIUS ANTUNES, PRA, PROJETO
 paes promete revitalizar prostibul: 	0.98
 uber lanca o uber boat exclusivame: 	0.98
 programa de governo de crivella in: 	0.98
 chefe de milicia vaidoso cria um m: 	0.98
 crivella promete renomear praia da: 	0.97
 corinthians e santos brigam por de: 	0.97
 levy fidelix diz que r

#### Exemplos de sátiras muito associadas a um tópico

In [ ]:
main_topic = []
mt_prob = []
for l in range(len(df['title'])):
  main_topic.append(doc_topic_matrix[l,:].argmax() + 1)
  mt_prob.append(doc_topic_matrix[l,:].max())

satires_s = df.assign(main_topic = main_topic, main_topic_prob = mt_prob)

In [ ]:
topico = 4
pd.options.display.max_colwidth = 300
satires_s[satires_s['main_topic'] == topico].sort_values('main_topic_prob', ascending = False)[['main_topic_prob','title', 'text']].sample(10)

In [ ]:
# Nomes das colunas
topicnames = ["Topic {}".format(i + 1) for i in range(doc_topic_matrix.shape[1])]

satire_topic = pd.DataFrame(np.round(doc_topic_matrix, 2), columns=topicnames, index = satires_s.index)
satire_topic[['title', 'text']] = satires_s[['title','text']]

ordem = ['title', 'text']
ordem.extend(topicnames)
letra_topico = satire_topic[ordem]

letra_topico.query('title == " jornais fecham acordo e virao com engov de brinde"').sort_values(by = 'Topic 8', ascending = False).head()

## Interpretando os tópicos!

## Tópico 1: Aqui encontramos sátiras relacionadas aos acontecimentos do governo do ex-presidente Michel Temer 

## Tópico 2: Sátiras que envolvem acontecimentos no estado do Rio de Janeiro

## Tópico 3: Envolve justiça e criminalidade

## Tópico 4: Envolve virais (vídeos, memes, acontecimentos) da Internet

## Tópico 5: Sátiras relacionadas aos acontecimentos do governo da ex-presidente Dilma Roussef ou que citam ela 

## Tópico 6: Sátiras relacionadas ao ex-presidente Lula e/ou ao deputado federal Aécio Neves 

## Tópico 7: Sátiras que envolvem acontecimentos no estado de São Paulo

## Tópico 8: Foge um pouco do contexto político e foca mais em situações cotidianas


